<a href="https://colab.research.google.com/github/skhabiri/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module3-Tune/LS_DS17_423_Tune_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [2]:
pip install wget

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import wget

In [4]:
def load_quickdraw10(path):
  wget.download(path)
  data = np.load('quickdraw10.npz')
  X = data['arr_0']
  y = data['arr_1']

  print(X.shape)
  print(y.shape)

  X, y = shuffle(X, y)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  
  return X_train, y_train, X_test, y_test

In [5]:
path = 'https://github.com/skhabiri/DS-Unit-4-Sprint-2-Neural-Networks/raw/main/module1-Architect/quickdraw10.npz'
X_train, y_train, X_test, y_test = load_quickdraw10(path)

(100000, 784)
(100000,)


In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

### Keras Tuner

In [7]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=2f4e84800ed137ab362f80081138b298945fdf0a39e623f58a5984d1034a515c
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=67a31b46b138eb73ff68d91d2cd814ba324f1207c5f1e5a290df0e88a6697db3
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [9]:
from tensorflow import keras
from tensorflow.keras import layers
import kerastuner.tuners as kt
from tensorflow.keras.optimizers import *

"""
This model Tunes:
- Number of Neurons in three Hidden Layers
- Learning Rate
- optimizer
- activation function
"""

def build_model(hp):
    hp_units1 = hp.Int('units_1', min_value=32, max_value=128, step=32)
    hp_lrate = hp.Choice('lrate', values = [1e-3]) 
    hp_optimizer = hp.Choice('optimizer', ['adam', 'rmsprop']) # 'sgd', 

    hp_activation = hp.Choice('activationfn',
                            [
                              # 'softmax',
                              # 'softplus',
                              # 'softsign',
                              'relu',
                              # 'tanh',
                              'sigmoid',
                              # 'hard_sigmoid',
                              'linear'
                            ])

    model = keras.Sequential()
    model.add(layers.Dense(units=hp_units1, input_dim=784, activation='relu'))
    for i in range(2,4,1):
      hp_units = hp.Int(
          'units_' + str(i),
          min_value=8,
          max_value=64,
          step=8
      )
      model.add(layers.Dense(units=hp_units, activation=hp_activation))
    model.add(layers.Dense(10, activation='softmax'))

    
    # if hp_optimizer == 'adam':
    #   opt = keras.optimizers.Adam(learning_rate=hp_lrate)
    # elif hp_optimizer== 'sgd':
    #   opt = keras.optimizers.SGD(learning_rate=hp_lrate)
    # elif hp_optimizer== 'rmsprop':
    #   opt = keras.optimizers.RMSprop(learning_rate=hp_lrate)
    # else:
    #   raise ValueError(f'Unexpected optimizer: {hp_optimizer}')
    
    opt_dic = {'adam': keras.optimizers.Adam(learning_rate=hp_lrate), 
            'sgd': keras.optimizers.SGD(learning_rate=hp_lrate),
            'rmsprop': keras.optimizers.RMSprop(learning_rate=hp_lrate)
            }

    model.compile(optimizer=opt_dic[hp_optimizer], loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [10]:
tuner_rs = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=8, 
                           executions_per_trial=5, directory='./kt-randomsearch', 
                           project_name='kt-RS')

In [11]:
tuner_rs.search_space_summary()

In [10]:
tuner_rs.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.5498 - accuracy: 0.5894 - val_loss: 0.9307 - val_accuracy: 0.7191
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.8227 - accuracy: 0.7496 - val_loss: 0.7753 - val_accuracy: 0.7731
Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.7426 - accuracy: 0.7787 - val_loss: 0.7122 - val_accuracy: 0.7884
Epoch 4/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.6902 - accuracy: 0.7950 - val_loss: 0.7163 - val_accuracy: 0.7893
Epoch 5/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.6467 - accuracy: 0.8074 - val_loss: 0.6817 - val_accuracy: 0.8007
Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.6410 - accuracy: 0.4587 - val_loss: 1.1773 - val_accuracy: 0.6000
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.0274 - accuracy: 0.6535 - val_loss: 0.9349 - val_accuracy: 0.6951

Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 2.3096 - accuracy: 0.1433 - val_loss: 2.1461 - val_accuracy: 0.1632
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 2.1402 - accuracy: 0.1675 - val_loss: 2.1028 - val_accuracy: 0.1792
Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 2.0104 - accuracy: 0.2140 - val_loss: 1.7834 - val_accuracy: 0.2826
Epoch 4/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.6029 - accuracy: 0.3979 - val_loss: 1.4132 - val_accuracy: 0.4635
Epoch 5/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.2267 - accuracy: 0.5755 - val_loss: 1.0903 - val_accuracy: 0.6726
Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 2.3292 - accuracy: 0.1113 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 2.3029 - accuracy: 0.0996 - val_loss: 2.3027 - val_accuracy: 0.0993

Epoch 1/5
2500/2500 [==============================] - 10s 4ms/step - loss: 1.1519 - accuracy: 0.6322 - val_loss: 0.8953 - val_accuracy: 0.7203
Epoch 2/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.8764 - accuracy: 0.7228 - val_loss: 0.8463 - val_accuracy: 0.7331
Epoch 3/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.8250 - accuracy: 0.7398 - val_loss: 0.7868 - val_accuracy: 0.7562
Epoch 4/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.7898 - accuracy: 0.7538 - val_loss: 0.7684 - val_accuracy: 0.7609
Epoch 5/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.7630 - accuracy: 0.7635 - val_loss: 0.7592 - val_accuracy: 0.7654
Epoch 1/5
2500/2500 [==============================] - 9s 4ms/step - loss: 1.1548 - accuracy: 0.6383 - val_loss: 0.8980 - val_accuracy: 0.7133
Epoch 2/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.8422 - accuracy: 0.7376 - val_loss: 0.7737 - val_accuracy: 0.760

Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 2.2046 - accuracy: 0.6797 - val_loss: 0.8161 - val_accuracy: 0.7806
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.7742 - accuracy: 0.7833 - val_loss: 0.7344 - val_accuracy: 0.8008
Epoch 3/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6719 - accuracy: 0.8067 - val_loss: 0.7274 - val_accuracy: 0.7968
Epoch 4/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6121 - accuracy: 0.8238 - val_loss: 0.6075 - val_accuracy: 0.8274
Epoch 5/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.5745 - accuracy: 0.8347 - val_loss: 0.6228 - val_accuracy: 0.8322
Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 2.1967 - accuracy: 0.6972 - val_loss: 0.8756 - val_accuracy: 0.7602
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.7606 - accuracy: 0.7867 - val_loss: 0.7021 - val_accuracy: 0.8000

Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 2.4475 - accuracy: 0.6247 - val_loss: 0.9885 - val_accuracy: 0.7337
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.9444 - accuracy: 0.7317 - val_loss: 0.9459 - val_accuracy: 0.7552
Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.8307 - accuracy: 0.7622 - val_loss: 0.7813 - val_accuracy: 0.7679
Epoch 4/5
2500/2500 [==============================] - 6s 3ms/step - loss: 0.7379 - accuracy: 0.7851 - val_loss: 0.7544 - val_accuracy: 0.7890
Epoch 5/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6831 - accuracy: 0.7997 - val_loss: 0.7719 - val_accuracy: 0.7793
Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 2.2771 - accuracy: 0.5911 - val_loss: 1.1542 - val_accuracy: 0.6322
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.9526 - accuracy: 0.7114 - val_loss: 0.8234 - val_accuracy: 0.7633

Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 2.7597 - accuracy: 0.6289 - val_loss: 0.8954 - val_accuracy: 0.7293
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.8585 - accuracy: 0.7477 - val_loss: 0.7950 - val_accuracy: 0.7620
Epoch 3/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.7592 - accuracy: 0.7740 - val_loss: 0.7444 - val_accuracy: 0.7841
Epoch 4/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6847 - accuracy: 0.7970 - val_loss: 0.6810 - val_accuracy: 0.8030
Epoch 5/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6331 - accuracy: 0.8115 - val_loss: 0.6946 - val_accuracy: 0.8015
Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 2.5820 - accuracy: 0.6354 - val_loss: 0.9977 - val_accuracy: 0.7272
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.8826 - accuracy: 0.7468 - val_loss: 0.8612 - val_accuracy: 0.7521

Epoch 1/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.7201 - accuracy: 0.5707 - val_loss: 1.0751 - val_accuracy: 0.6899
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.9907 - accuracy: 0.7113 - val_loss: 0.9180 - val_accuracy: 0.7283
Epoch 3/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.8488 - accuracy: 0.7551 - val_loss: 0.7769 - val_accuracy: 0.7803
Epoch 4/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.7685 - accuracy: 0.7799 - val_loss: 0.7591 - val_accuracy: 0.7888
Epoch 5/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.7315 - accuracy: 0.7888 - val_loss: 0.7222 - val_accuracy: 0.8007
Epoch 1/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.7187 - accuracy: 0.5508 - val_loss: 1.1856 - val_accuracy: 0.6666
Epoch 2/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.0161 - accuracy: 0.7110 - val_loss: 0.9450 - val_accuracy: 0.7439

Epoch 1/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.9566 - accuracy: 0.4793 - val_loss: 1.3072 - val_accuracy: 0.5662
Epoch 2/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.2491 - accuracy: 0.5978 - val_loss: 1.1474 - val_accuracy: 0.6155
Epoch 3/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.1020 - accuracy: 0.6532 - val_loss: 1.3252 - val_accuracy: 0.6210
Epoch 4/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.9285 - accuracy: 0.7192 - val_loss: 0.8474 - val_accuracy: 0.7495
Epoch 5/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.8043 - accuracy: 0.7669 - val_loss: 0.7925 - val_accuracy: 0.7756
Epoch 1/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.9147 - accuracy: 0.4594 - val_loss: 1.4715 - val_accuracy: 0.5079
Epoch 2/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.3187 - accuracy: 0.5549 - val_loss: 1.2603 - val_accuracy: 0.6083

INFO:tensorflow:Oracle triggered exit


In [11]:
tuner_rs.results_summary()

In [12]:
best_model = tuner_rs.get_best_models()[0]
# Evaluate the best model.
loss0, accuracy0 = best_model.evaluate(X_test, y_test)
print(f"""best accuracy: {accuracy0}""")
print("best parameters", tuner_rs.get_best_hyperparameters(num_trials=1)[0].values)

625/625 [==============================] - 1s 1ms/step - loss: 0.5916 - accuracy: 0.8343
best accuracy: 0.8343499898910522
best parameters {'units_1': 96, 'lrate': 0.001, 'optimizer': 'rmsprop', 'activationfn': 'linear', 'units_2': 16, 'units_3': 16}


In [16]:
tuner_hb = kt.Hyperband(build_model,
                     objective = 'val_accuracy', 
                     max_epochs = 5,
                     factor = 3,
                     directory = './kt-hyperbrand',
                     project_name = 'kt-HB')  

In [17]:
tuner_hb.search_space_summary()

In [18]:
tuner_hb.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/2
2500/2500 [==============================] - 6s 2ms/step - loss: 2.4144 - accuracy: 0.0987 - val_loss: 2.3045 - val_accuracy: 0.1014
Epoch 2/2
2500/2500 [==============================] - 6s 2ms/step - loss: 2.3041 - accuracy: 0.1003 - val_loss: 2.3042 - val_accuracy: 0.1033


Epoch 1/2
2500/2500 [==============================] - 5s 2ms/step - loss: 2.4606 - accuracy: 0.1591 - val_loss: 1.7741 - val_accuracy: 0.3076
Epoch 2/2
2500/2500 [==============================] - 5s 2ms/step - loss: 1.7791 - accuracy: 0.3154 - val_loss: 1.9851 - val_accuracy: 0.2465


Epoch 1/2
2500/2500 [==============================] - 5s 2ms/step - loss: 1.8963 - accuracy: 0.4676 - val_loss: 1.5312 - val_accuracy: 0.6213
Epoch 2/2
2500/2500 [==============================] - 5s 2ms/step - loss: 1.3055 - accuracy: 0.6639 - val_loss: 1.1376 - val_accuracy: 0.6920


Epoch 1/2
2500/2500 [==============================] - 6s 2ms/step - loss: 1.3643 - accuracy: 0.5407 - val_loss: 1.1243 - val_accuracy: 0.6266
Epoch 2/2
2500/2500 [==============================] - 6s 2ms/step - loss: 1.0554 - accuracy: 0.6533 - val_loss: 0.9999 - val_accuracy: 0.6895


Epoch 1/2
2500/2500 [==============================] - 6s 2ms/step - loss: 1.9457 - accuracy: 0.4457 - val_loss: 1.6446 - val_accuracy: 0.5642
Epoch 2/2
2500/2500 [==============================] - 6s 2ms/step - loss: 1.4357 - accuracy: 0.6171 - val_loss: 1.2624 - val_accuracy: 0.6583


...

Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.2929 - accuracy: 0.5850 - val_loss: 1.0961 - val_accuracy: 0.6298
Epoch 4/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.9965 - accuracy: 0.6783 - val_loss: 0.9959 - val_accuracy: 0.6909
Epoch 5/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.9432 - accuracy: 0.6978 - val_loss: 0.9523 - val_accuracy: 0.7019


Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.4926 - accuracy: 0.4969 - val_loss: 1.1559 - val_accuracy: 0.6207
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.0346 - accuracy: 0.6652 - val_loss: 0.9543 - val_accuracy: 0.6970
Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.9046 - accuracy: 0.7116 - val_loss: 0.8873 - val_accuracy: 0.7172
Epoch 4/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.8427 - accuracy: 0.7345 - val_loss: 0.8237 - val_accuracy: 0.7430
Epoch 5/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.7995 - accuracy: 0.7503 - val_loss: 0.8043 - val_accuracy: 0.7452


Epoch 1/5
2500/2500 [==============================] - 4s 2ms/step - loss: 5.3217 - accuracy: 0.3071 - val_loss: 1.7276 - val_accuracy: 0.4069
Epoch 2/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.5417 - accuracy: 0.4539 - val_loss: 1.3923 - val_accuracy: 0.5066
Epoch 3/5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.2840 - accuracy: 0.5362 - val_loss: 1.2251 - val_accuracy: 0.5771
Epoch 4/5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.1382 - accuracy: 0.5964 - val_loss: 1.1224 - val_accuracy: 0.6101
Epoch 5/5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.0466 - accuracy: 0.6371 - val_loss: 1.0608 - val_accuracy: 0.6460


Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.7812 - accuracy: 0.3093 - val_loss: 1.6587 - val_accuracy: 0.4119
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.6409 - accuracy: 0.3873 - val_loss: 1.7407 - val_accuracy: 0.3380
Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.6315 - accuracy: 0.3758 - val_loss: 1.5662 - val_accuracy: 0.4170
Epoch 4/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.5529 - accuracy: 0.4104 - val_loss: 1.6680 - val_accuracy: 0.3739
Epoch 5/5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.6872 - accuracy: 0.3716 - val_loss: 1.7278 - val_accuracy: 0.3386


INFO:tensorflow:Oracle triggered exit


In [19]:
tuner_hb.get_best_hyperparameters(num_trials=1)[0].values

{'activationfn': 'tanh',
 'lrate': 0.0001,
 'tuner/bracket': 0,
 'tuner/epochs': 5,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_1': 96,
 'units_2': 56,
 'units_3': 48}

In [20]:
# Evaluate the best model.
print("best accuracy: ", tuner_hb.get_best_models()[0].evaluate(X_test, y_test)[1])
print("best parameters", tuner_hb.get_best_hyperparameters(num_trials=1)[0].values)

625/625 [==============================] - 1s 1ms/step - loss: 0.8043 - accuracy: 0.7452
best accuracy:  0.7452499866485596
best parameters {'units_1': 96, 'lrate': 0.0001, 'activationfn': 'tanh', 'units_2': 56, 'units_3': 48, 'tuner/epochs': 5, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


### While Hyperbrand runs faster, RandomSearch tuner does a better job in finding the optimum hyper parameters.

### Experiment Tracking Framework using tensorboard

In [12]:
# Load an ipython extension
%load_ext tensorboard

In [13]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import os
import datetime

In [20]:
# Define hyper parametres and score metrics
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([32,64]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([8,32]))

HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.005))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_ACTIVATION = hp.HParam('actfn', hp.Discrete(['relu', 'sigmoid']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS1,HP_NUM_UNITS2, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

In [21]:
# Adapt Model Function with HParams
def train_test_model(hparams):
  
  model = tf.keras.Sequential(
      [layers.Dense(units=hparams[HP_NUM_UNITS1], input_dim=784, activation='relu'),
       layers.Dense(units=hparams[HP_NUM_UNITS2], activation=hparams[HP_ACTIVATION]),
       layers.Dense(10, activation='softmax')
       ])

  # Optimizer need the learning rate
  opt_name = hparams[HP_OPTIMIZER]
  lr = hparams[HP_LEARNING_RATE]

  if opt_name == 'adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
  elif opt_name == 'sgd':
    opt = tf.keras.optimizers.SGD(learning_rate=lr)
  else:
    raise ValueError(f'Unexpected optimizer: {opt_name}')

  # Compile defines optimizer, loss function and metric
  model.compile(
      optimizer=opt,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(X_train, y_train, epochs=5)
  _, accuracy = model.evaluate(X_test, y_test)

  return accuracy

In [25]:
# For each run, log an hparams set and final accuracy
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    # .hparams is a method of hp
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [24]:
# Loop over parameters to create unique params set
session_num = 0

# Basically a grid search
for num_units1 in HP_NUM_UNITS1.domain.values:
  for num_units2 in HP_NUM_UNITS2.domain.values:
    for learning_rate in (HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value):
      for activation in HP_ACTIVATION.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
          # create a dict eith key of type hp.HParam and unique values , grid search
          hparams = {
            HP_NUM_UNITS1: num_units1,
            HP_NUM_UNITS2: num_units2,
            HP_LEARNING_RATE: learning_rate,
            HP_ACTIVATION: activation,
            HP_OPTIMIZER: optimizer
            }

          run_name = f'run-{session_num}'
          print(f'--- Starting trial: {run_name}')
          # type(param): <class 'tensorboard.plugins.hparams.summary_v2.HParam'>
          # param: <HParam 'num_units1': {32,64,128}>
          #  param.name: num_units1
          print({param.name: hparams[param] for param in hparams})
          run('logs/hparams_tuning/' + run_name, hparams)
          session_num += 1

--- Starting trial: run-0
{'num_units1': 32, 'num_units2': 8, 'learning_rate': 0.001, 'actfn': 'linear', 'optimizer': 'adam'}
Epoch 1/5
2500/2500 [==============================] - 4s 2ms/step - loss: 2.7428 - accuracy: 0.3351
Epoch 2/5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.4422 - accuracy: 0.4659
Epoch 3/5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.1362 - accuracy: 0.6009
Epoch 4/5
2500/2500 [==============================] - 4s 2ms/step - loss: 0.9959 - accuracy: 0.6632
Epoch 5/5
625/625 [==============================] - 1s 1ms/step - loss: 0.9682 - accuracy: 0.7005
--- Starting trial: run-1
{'num_units1': 32, 'num_units2': 8, 'learning_rate': 0.001, 'actfn': 'linear', 'optimizer': 'sgd'}
Epoch 1/5
2500/2500 [==============================] - 3s 1ms/step - loss: 2.2101 - accuracy: 0.2773
Epoch 2/5
2500/2500 [==============================] - 3s 1ms/step - loss: 1.6366 - accuracy: 0.3932
Epoch 3/5
2500/2500 [=======================

In [26]:
# Visualization with tensorboard
%tensorboard --logdir logs/hparams_tuning

<IPython.core.display.Javascript object>

### Scikit learn hypertunning tools: GridSearchCV, RandomizedSearchCV

In [ ]:
# WARNING - may take a few minutes before any output is visible

import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# relu is fast for computation. 0 or y=x, compared to sigmoid

# Function to create model with .fit method. This is required in KerasClassifier

def create_model(units1=32, units2=8, opti='rmsprop', init='glorot_uniform', actfn='relu'):
    # create model
    model = Sequential()
    # units are the number of hidden neurons
    model.add(Dense(units1, input_dim=784, kernel_initializer=init, activation='relu'))
    model.add(Dense(units2, kernel_initializer=init, activation=actfn))
    # 10 output labels
    model.add(Dense(10, kernel_initializer=init, activation='softmax'))
    # Compile model
    #sparse requires 1hot-encoding at the output
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opti, metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model that can accept the fit method
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
optimizers = ['rmsprop', 'adam']
optimizer = {'rmsprop': keras.optimizers.RMSprop(learn_rate),
             'adam': keras.optimizers.Adam(learn_rate)
             }

lrates = [0.001]
inits = ['normal', 'uniform']
epochs = [5, 10]
batches = [32, 128]
acts = ['relu', 'sigmoid']
# param_grid = dict(opti=optimizers, epochs=epochs, batch_size=batches, init=inits, units1=[32, 64], units2=[8,32], actfn=acts)

param_grid = {'batch_size': batches,
              'epochs': epochs,
              'units1': [32, 64],
              'units2': [8, 32],
              'opti': optimizer[opimizers],
              'learn_rate' : lrates,
              'init': inits
              }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

# Cross validation parameters
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
# one mean, std, param set for each gridsearch vector. 
# The mean refers to test_score mean of all epochs within each gridsearch verctor
# The epochs log only shows the epoch train steps for the best gridsearch vector (parameter set), batch_size=32 in this case
# perhaps gridsearch decide the best params based on mean score of all epochs, which is not necessarily the best last trained score
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 